# Dados de Entrada
* Link: https://tinyurl.com/bigdata-gut-pt
* Selecione "Adicionar ao Drive"





# Setup

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!apt-get update  > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install findspark pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=272d595a586e8d3b921666d757feda9693e57b5e7be5a375b08dbef54062993a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.5.1-bin-hadoop3

env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.5.1-bin-hadoop3


In [4]:
import findspark
findspark.init("/content/spark-3.5.1-bin-hadoop3")

In [5]:
from pyspark.sql import SparkSession

from datetime import datetime

appName = 'Big Data'
master = 'local'

spark = SparkSession.builder     \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Entradas de teste e validação

Os comandos abaixo criam duas entradas para desenvolvimento e validação no diretório `entradas_teste`:

*   Arquivo A.txt: "Um carro está dirigindo na rua."
*   Arquivo B.txt: "Um caminhão está dirigindo na rodovia."
*   Arquivo C.txt: "Um motorista encontrou com outro motorista."

O resultado esperado para a etapa de cálculo de TF para estes arquivos é:

```
[(('A.txt', 'um'), 0.16666666666666666),
 (('A.txt', 'carro'), 0.16666666666666666),
 (('A.txt', 'está'), 0.16666666666666666),
 (('A.txt', 'dirigindo'), 0.16666666666666666),
 (('A.txt', 'na'), 0.16666666666666666),
 (('A.txt', 'rua'), 0.16666666666666666),
 (('B.txt', 'um'), 0.16666666666666666),
 (('B.txt', 'caminhão'), 0.16666666666666666),
 (('B.txt', 'está'), 0.16666666666666666),
 (('B.txt', 'dirigindo'), 0.16666666666666666),
 (('B.txt', 'na'), 0.16666666666666666),
 (('B.txt', 'rodovia'), 0.16666666666666666),
 (('C.txt', 'um'), 0.16666666666666666),
 (('C.txt', 'motorista'), 0.3333333333333333),
 (('C.txt', 'encontrou'), 0.16666666666666666),
 (('C.txt', 'com'), 0.16666666666666666),
 (('C.txt', 'outro'), 0.16666666666666666)]
 ```

 O resultado esperado para o cálculo de IDF é:

````
[('um', 0.0),
 ('carro', 1.0986122886681098),
 ('está', 0.4054651081081644),
 ('dirigindo', 0.4054651081081644),
 ('na', 0.4054651081081644),
 ('rua', 1.0986122886681098),
 ('caminhão', 1.0986122886681098),
 ('rodovia', 1.0986122886681098),
 ('motorista', 1.0986122886681098),
 ('encontrou', 1.0986122886681098),
 ('com', 1.0986122886681098),
 ('outro', 1.0986122886681098)]
 ````

Finalmente, o resultado final (TD IDF ordenado) esperado é:

```
[(('C.txt', 'motorista'), 0.37),
 (('B.txt', 'caminhão'), 0.18),
 (('B.txt', 'rodovia'), 0.18),
 (('C.txt', 'encontrou'), 0.18),
 (('C.txt', 'com'), 0.18),
 (('C.txt', 'outro'), 0.18),
 (('A.txt', 'carro'), 0.18),
 (('A.txt', 'rua'), 0.18),
 (('A.txt', 'está'), 0.07),
 (('B.txt', 'está'), 0.07),
 (('A.txt', 'dirigindo'), 0.07),
 (('B.txt', 'dirigindo'), 0.07),
 (('A.txt', 'na'), 0.07),
 (('B.txt', 'na'), 0.07),
 (('A.txt', 'um'), 0.0),
 (('B.txt', 'um'), 0.0),
 (('C.txt', 'um'), 0.0)]
```

Use estas entradas para desenvolvimento, testes, e validação. Para a solução final, processe todos os arquivos no diretório `all` do pacote de dados.


In [6]:
!mkdir entradas_teste

In [7]:
!rm entradas_teste/*

rm: cannot remove 'entradas_teste/*': No such file or directory


In [8]:
!echo "Um carro está dirigindo na rua." >> entradas_teste/A.txt

In [9]:
!echo "Um caminhão está dirigindo na rodovia." >> entradas_teste/B.txt

In [10]:
!echo "Um motorista encontrou com outro motorista." >> entradas_teste/C.txt

# Solução

In [11]:
input_dir = "/content/entradas_teste/"
input_files = spark.sparkContext.wholeTextFiles(input_dir + "*")

In [108]:
input_files.take(3)

[('file:/content/entradas_teste/A.txt', 'Um carro está dirigindo na rua.\n'),
 ('file:/content/entradas_teste/B.txt',
  'Um caminhão está dirigindo na rodovia.\n'),
 ('file:/content/entradas_teste/C.txt',
  'Um motorista encontrou com outro motorista.\n')]

In [142]:
D = input_files.count()

In [22]:
import re

def process_file(file):

    filename, text = file[0], file[1]

    filename = re.sub(input_dir, "", filename)

    text = re.sub("\*End of .*Project Gutenberg.*", "", text, flags=re.IGNORECASE|re.DOTALL)
    text = re.sub("[^a-zà-ù ]", " ", text.lower())

    words = text.split()
    for w in words :
        yield ((filename, w), 1 / len(words))

In [23]:
wp = input_files.flatMap(process_file)

In [24]:
wp.take(2)

[(('file:A.txt', 'um'), 0.16666666666666666),
 (('file:A.txt', 'carro'), 0.16666666666666666)]

In [25]:
wtf = wp.reduceByKey(lambda acc, value: acc + value)

In [26]:
wtf.take(20)

[(('file:A.txt', 'um'), 0.16666666666666666),
 (('file:A.txt', 'carro'), 0.16666666666666666),
 (('file:A.txt', 'está'), 0.16666666666666666),
 (('file:A.txt', 'dirigindo'), 0.16666666666666666),
 (('file:A.txt', 'na'), 0.16666666666666666),
 (('file:A.txt', 'rua'), 0.16666666666666666),
 (('file:B.txt', 'um'), 0.16666666666666666),
 (('file:B.txt', 'caminhão'), 0.16666666666666666),
 (('file:B.txt', 'está'), 0.16666666666666666),
 (('file:B.txt', 'dirigindo'), 0.16666666666666666),
 (('file:B.txt', 'na'), 0.16666666666666666),
 (('file:B.txt', 'rodovia'), 0.16666666666666666),
 (('file:C.txt', 'um'), 0.16666666666666666),
 (('file:C.txt', 'motorista'), 0.3333333333333333),
 (('file:C.txt', 'encontrou'), 0.16666666666666666),
 (('file:C.txt', 'com'), 0.16666666666666666),
 (('file:C.txt', 'outro'), 0.16666666666666666)]

In [27]:
def process_file_adjusted(file):

    filename, text = file[0], file[1]

    filename = re.sub(input_dir, "", filename)

    text = re.sub("\*End of .*Project Gutenberg.*", "", text, flags=re.IGNORECASE|re.DOTALL)
    text = re.sub("[^a-zà-ù ]", " ", text.lower())

    words = text.split()
    for w in words :
        yield (w, filename)

In [28]:
term_file = input_files.flatMap(process_file_adjusted)

In [30]:
term_file.take(10)

[('um', 'file:A.txt'),
 ('carro', 'file:A.txt'),
 ('está', 'file:A.txt'),
 ('dirigindo', 'file:A.txt'),
 ('na', 'file:A.txt'),
 ('rua', 'file:A.txt'),
 ('um', 'file:B.txt'),
 ('caminhão', 'file:B.txt'),
 ('está', 'file:B.txt'),
 ('dirigindo', 'file:B.txt')]

In [112]:
def agg_element(acc, file):
    return acc.union({file})

def agg_partials(acc1, acc2):
    return None # acc1.union(acc2)

term_file_agg = term_file.aggregateByKey(set(), agg_element, agg_partials)
term_file_agg.take(20)

[('um', {'file:A.txt', 'file:B.txt', 'file:C.txt'}),
 ('carro', {'file:A.txt'}),
 ('está', {'file:A.txt', 'file:B.txt'}),
 ('dirigindo', {'file:A.txt', 'file:B.txt'}),
 ('na', {'file:A.txt', 'file:B.txt'}),
 ('rua', {'file:A.txt'}),
 ('caminhão', {'file:B.txt'}),
 ('rodovia', {'file:B.txt'}),
 ('motorista', {'file:C.txt'}),
 ('encontrou', {'file:C.txt'}),
 ('com', {'file:C.txt'}),
 ('outro', {'file:C.txt'})]

In [113]:
term_file_count = term_file_agg.mapValues(len)
term_file_count.take(20)

[('um', 3),
 ('carro', 1),
 ('está', 2),
 ('dirigindo', 2),
 ('na', 2),
 ('rua', 1),
 ('caminhão', 1),
 ('rodovia', 1),
 ('motorista', 1),
 ('encontrou', 1),
 ('com', 1),
 ('outro', 1)]

In [136]:
term_file_partial = term_file_agg.join(term_file_count)
term_file_partial.take(20)

[('um', ({'file:A.txt', 'file:B.txt', 'file:C.txt'}, 3)),
 ('carro', ({'file:A.txt'}, 1)),
 ('está', ({'file:A.txt', 'file:B.txt'}, 2)),
 ('dirigindo', ({'file:A.txt', 'file:B.txt'}, 2)),
 ('na', ({'file:A.txt', 'file:B.txt'}, 2)),
 ('rua', ({'file:A.txt'}, 1)),
 ('caminhão', ({'file:B.txt'}, 1)),
 ('rodovia', ({'file:B.txt'}, 1)),
 ('motorista', ({'file:C.txt'}, 1)),
 ('encontrou', ({'file:C.txt'}, 1)),
 ('com', ({'file:C.txt'}, 1)),
 ('outro', ({'file:C.txt'}, 1))]

In [137]:
term_file = term_file_partial.map(lambda value: (value[0], value[1][1]))
term_file.take(20)

[('um', 3),
 ('carro', 1),
 ('está', 2),
 ('dirigindo', 2),
 ('na', 2),
 ('rua', 1),
 ('caminhão', 1),
 ('rodovia', 1),
 ('motorista', 1),
 ('encontrou', 1),
 ('com', 1),
 ('outro', 1)]

In [138]:
import math

w_idf = term_file.mapValues(lambda value: math.log(D / value))

In [123]:
w_idf.take(20)

[('um', 0.0),
 ('carro', 1.0986122886681098),
 ('está', 0.4054651081081644),
 ('dirigindo', 0.4054651081081644),
 ('na', 0.4054651081081644),
 ('rua', 1.0986122886681098),
 ('caminhão', 1.0986122886681098),
 ('rodovia', 1.0986122886681098),
 ('motorista', 1.0986122886681098),
 ('encontrou', 1.0986122886681098),
 ('com', 1.0986122886681098),
 ('outro', 1.0986122886681098)]

In [124]:
0.3333333333333333 * 1.0986122886681098

0.3662040962227032

In [125]:
wtf.take(3)

[(('file:A.txt', 'um'), 0.16666666666666666),
 (('file:A.txt', 'carro'), 0.16666666666666666),
 (('file:A.txt', 'está'), 0.16666666666666666)]

In [126]:
wtf_adjusted = wtf.map(lambda value: (value[0][1], (value[0][0], value[1])))

In [127]:
wtf_adjusted.take(1)

[('um', ('file:A.txt', 0.16666666666666666))]

In [139]:
w_tfidf = wtf_adjusted.join(w_idf)

In [129]:
w_tfidf.take(1)

[('está', (('file:A.txt', 0.16666666666666666), 0.4054651081081644))]

In [80]:
0.16666666666666666 * 0.4054651081081644

0.06757751801802739

In [130]:
w_tfidf_adjusted = w_tfidf.map(lambda value: ( (value[1][0][0], value[0] ), value[1][0][1] * value[1][1]) )

In [131]:
tfidf_sorted = w_tfidf_adjusted.mapValues(lambda value: round(value, 2))\
                    .sortBy(lambda value: value[1], ascending=False)

In [132]:
tfidf_sorted.collect()

[(('file:C.txt', 'motorista'), 0.37),
 (('file:B.txt', 'caminhão'), 0.18),
 (('file:B.txt', 'rodovia'), 0.18),
 (('file:C.txt', 'encontrou'), 0.18),
 (('file:C.txt', 'com'), 0.18),
 (('file:C.txt', 'outro'), 0.18),
 (('file:A.txt', 'carro'), 0.18),
 (('file:A.txt', 'rua'), 0.18),
 (('file:A.txt', 'está'), 0.07),
 (('file:B.txt', 'está'), 0.07),
 (('file:A.txt', 'dirigindo'), 0.07),
 (('file:B.txt', 'dirigindo'), 0.07),
 (('file:A.txt', 'na'), 0.07),
 (('file:B.txt', 'na'), 0.07),
 (('file:A.txt', 'um'), 0.0),
 (('file:B.txt', 'um'), 0.0),
 (('file:C.txt', 'um'), 0.0)]

In [140]:
# Conjunto completo (para solução final)
input_dir = "/content/drive/MyDrive/Colab Notebooks/gut-pt/all/"

# Conjunto menor (para desenvolvimento)
small_input_dir = "/content/drive/MyDrive/Colab Notebooks/gut-pt/small/"

# Entradas teste (para testes e verificação)
test_dir = "/content/entradas_teste/"

In [143]:
import re
import math


def process_file(file):

    filename, text = file[0], file[1]

    filename = re.sub(input_dir, "", filename)

    text = re.sub("\*End of .*Project Gutenberg.*", "", text, flags=re.IGNORECASE|re.DOTALL)
    text = re.sub("[^a-zà-ù ]", " ", text.lower())

    words = text.split()
    for w in words :
        yield ((filename, w), 1 / len(words))

def process_file_adjusted(file):

    filename, text = file[0], file[1]

    filename = re.sub(input_dir, "", filename)

    text = re.sub("\*End of .*Project Gutenberg.*", "", text, flags=re.IGNORECASE|re.DOTALL)
    text = re.sub("[^a-zà-ù ]", " ", text.lower())

    words = text.split()
    for w in words :
        yield (w, filename)

def agg_element(acc, file):
    return acc.union({file})

def agg_partials(acc1, acc2):
    return None

In [144]:
input_files = spark.sparkContext.wholeTextFiles(small_input_dir+"*")
D = input_files.count()

In [145]:
wp = input_files.flatMap(process_file)
wtf = wp.reduceByKey(lambda acc, value: acc + value)
term_file = input_files.flatMap(process_file_adjusted)

term_file_agg = term_file.aggregateByKey(set(), agg_element, agg_partials)
term_file_count = term_file_agg.mapValues(len)
term_file_partial = term_file_agg.join(term_file_count)
term_file = term_file_partial.map(lambda value: (value[0], value[1][1]))


w_idf = term_file.mapValues(lambda value: math.log(D / value))

wtf_adjusted = wtf.map(lambda value: (value[0][1], (value[0][0], value[1])))

w_tfidf = wtf_adjusted.join(w_idf)
w_tfidf_adjusted = w_tfidf.map(lambda value: ( (value[1][0][0], value[0] ), value[1][0][1] * value[1][1]) )

tfidf_sorted = w_tfidf_adjusted.mapValues(lambda value: round(value, 2))\
                    .sortBy(lambda value: value[1], ascending=False)

In [146]:
tfidf_sorted.take(10)

[(('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-55682-8',
   'rubião'),
  0.01),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-27350-8',
   'gutenberg'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-29040-8',
   'gutenberg'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-33056-8',
   'gutenberg'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-54829-8',
   'gutenberg'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-55682-8',
   'gutenberg'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-27350-8',
   'ebook'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-29040-8',
   'ebook'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-33056-8',
   'ebook'),
  0.0),
 (('file:/content/drive/MyDrive/Colab Notebooks/gut-pt/small/u-54829-8',
   'ebook'),
  0.0)]